In [0]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

In [0]:
with open('anna.txt','r') as f:
  text=f.read()
  vocab=sorted(set(text))
  vocab_to_int={c:i for i,c in enumerate(vocab)}
  int_to_vocab=dict(enumerate(vocab))
  encoded=np.array([vocab_to_int[c] for c in text],dtype=np.int32)

In [0]:
print(text[:100])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin


In [0]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70],
      dtype=int32)

Type of a Classification Problem:
      num_classes=len()

In [0]:
len(vocab)

83

##Mini-Batches

In [0]:
def get_batches(arr,batch_size,n_steps):
  characters_per_batch=batch_size*n_steps
  n_batches=len(arr)//characters_per_batch
  
  arr=arr[:(characters_per_batch*n_batches)]
  
  arr=arr.reshape(batch_size,n_steps*n_batches)
  
  for n in range(0,arr.shape[1],n_steps):
    x=arr[:,n:n+n_steps]
    y_temp=arr[:,n+1:n+n_steps+1]#but what about last batch
    y=np.zeros(x.shape,dtype=x.dtype)
    y[:,:y_temp.shape[1]]=y_temp #adds appropriate number of zeros in last batch
    yield x,y

In [0]:
batches=get_batches(encoded,10,50)
x,y=next(batches)

In [0]:
int_to_vocab[0]

'\n'

In [0]:
print('x\n',x[:10,:10])
print('y\n',y[:10,:10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]
y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


In [0]:
def build_inputs(batch_size,num_steps):
  inputs=tf.placeholder(tf.int32,[batch_size,num_steps],name='inputs')
  targets=tf.placeholder(tf.int32,[batch_size,num_steps],name='targets')
  keep_prob=tf.placeholder(tf.float32,name='keep_prob')
  return inputs,targets,keep_prob

In [0]:
def build_lstm(lstm_size,num_layers,batch_size,keep_prob):
  
  def build_cell(lstm_size,keep_prob):
    lstm=tf.nn.rnn_cell.LSTMCell(num_units=lstm_size)
    drop=tf.nn.rnn_cell.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    return drop
  cell=tf.nn.rnn_cell.MultiRNNCell([build_cell(lstm_size,keep_prob) for _ in range(num_layers)])
  initial_state=cell.zero_state(batch_size,tf.float32)
  return cell,initial_state

In [0]:
def build_output(lstm_output,in_size,out_size):
  seq_output=tf.concat(lstm_output,axis=1)
  x=tf.reshape(seq_output,[-1,in_size])

  with tf.variable_scope('softmax'):
    softmax_w=tf.Variable(tf.truncated_normal((in_size,out_size),stddev=0.1))
    softmax_b=tf.Variable(tf.zeros(out_size))
    
    logits=tf.matmul(x,softmax_w)+softmax_b
    
    out=tf.nn.softmax(logits,name='predictions')
    
    return out,logits

In [0]:
def build_loss(logits,targets,lstm_size,num_classes):
  y_one_hot=tf.one_hot(targets,num_classes)
  y_reshaped=tf.reshape(y_one_hot,logits.get_shape())
  
  loss=tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
  loss=tf.reduce_mean(loss)
  return loss

In [0]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [0]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01   # Learning rate
keep_prob = 0.5

In [0]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [0]:
epochs = 40
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/40...  Training Step: 50...  Training loss: 3.0755...  0.1545 sec/batch
Epoch: 1/40...  Training Step: 100...  Training loss: 2.5995...  0.1529 sec/batch
Epoch: 1/40...  Training Step: 150...  Training loss: 2.3692...  0.1541 sec/batch
Epoch: 2/40...  Training Step: 200...  Training loss: 2.1992...  0.1578 sec/batch
Epoch: 2/40...  Training Step: 250...  Training loss: 2.0792...  0.1562 sec/batch
Epoch: 2/40...  Training Step: 300...  Training loss: 1.9229...  0.1561 sec/batch
Epoch: 2/40...  Training Step: 350...  Training loss: 1.8741...  0.1524 sec/batch
Epoch: 3/40...  Training Step: 400...  Training loss: 1.7467...  0.1533 sec/batch
Epoch: 3/40...  Training Step: 450...  Training loss: 1.6733...  0.1574 sec/batch
Epoch: 3/40...  Training Step: 500...  Training loss: 1.6132...  0.1541 sec/batch
Epoch: 3/40...  Training Step: 550...  Training loss: 1.5878...  0.1564 sec/batch
Epoch: 4/40...  Training Step: 600...  Training loss: 1.5056...  0.1537 sec/batch
Epoch: 4/40...  T

In [0]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i7920_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3200_l512.ckpt"
all_model_checkpoint_pa

In [0]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [0]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [0]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i7920_l512.ckpt'

In [0]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Far too happy,
sometimes, the second same word. Sergey Ivanovitch's attitude to my
husband, and was always serious one. And at that moment, he could not
torching his feigher to be so awful and talking. And he was in an
insult to her; all he wanted to say that it would be so much as so struggles
in them into that servant, and all had been a stronger friends with
his first arrangement.

Then he was not saying that it was awaiting herself in him that how he
was something and had time to be alone. She did not say that he could not
tell the same, and he could not have said, and the paint had an association
that she was thinking.

"You will never have sooned things and then and the doctor, and I will
beal to myself," said the princess. All the clight of the stations of
their attitude, and the commenculated stuffing at a caressing of action,
she felt at a man, and the same, with the candiduties of the sight
of his feeling of accordance as he said, she was talking to her. He
stared at her son 

In [0]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Far s whet har ans
on shit she che wers the wer that
hat whe sass of wer sam the
the hav on he wat hat and and at of tho shat his,"
sand sow the corser tere her as had, ald him went ant the pind, at the she the somtant and the
sand wink the pith the was the
galy wish and her so dit ho dessers, wor had
wass, sho canles wouk, she has would shers of the pot time hers to tiste hord an to him to the wels was a de that his whonge she tor to se shither she wass he tong and will had, wat her and thom, stout tho the
prace an her he wise he was him
astols and the cas to the sass the sors he wort te that he sat he the se tit the wind
the co th at a pon werting whe sond on
his sill with he se tham the what ald the were and hat a dads and hid, and the hers tore the pots on the she songhered whet his as the had him
sand, and the sad so whin wisted at wime and and he wamle a sow te weme the sald
the should was tors at thime a sout of he tow
wom to drind, and her than he whet the were sise was
ang the

In [0]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far the once")
print(samp)

Far the once, and she was not in the position to anywered in
the servinch at him, and winter hand.

Stryou saw all a chattion with anyone of
the meatings. "I am shalt the some only
to the must was in told him?"

"I say that this asticing that serving home."

The sour. She seived at a stalk of her was and a convisious to
the more to the pare a stopped that she
was there was than a feeling had
been them, and were
to glance has
step the mosticition of the possion.

The sain it, and the madine, sat that had setting to his hands about something of the propling when himself that her same as a mon in the serval,
want officure.

"I would not speek and an too."

"Well, what have she's thought with him out of the
shalled
of the cordier. He don't gres, I have not askow all his sighel stards of this howen and stall him of himself of the placile they heard seeing tomeress, the persons. There have think of howers. This say
to
the mast on
this was still been she
could not saw, at the
constant of his,

In [0]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farvata over the most course the
mother, still te hear of the part, and
somewhing astachering, that the master was almost to show the carriage with the work this he would not consider a smile, and
with her satisfactes, with a finish of the marriage, and a from with a little compance, they
was and was a share as they went to take this women was the statefully as to the position which had been words, though
it were to have taken into the manself, and shreek him with
themself as intolerately so intensed, to thong, which set to
the conversation to the prince shatched her hand. She had no despit men her sisters, he felt he came at
her single, as he would be so fond about their sacrearing in the mother, and they went
out.

"Only that's a people think is the might see it all the position in her and there will say them. He wanted to too
to hear, how if you said that I am
not tone in the seet of their man, when I want to
hear to string that it is a finging to the pluce of her simples than all, 